# TP6 : Diffie-Hellman et logarithme discret

Soit $G$ un groupe cyclique d'ordre n, et $g \in G$ un générateur de $G$. 

En informatique, il est très facile, étant donné $n \in \mathbb N$ de calculer $g ^n$ (voir ci-dessous). Mais si $\alpha \in G$; il est à l'inverse très difficile de trouver un $n$ tel que $\alpha = g^n$. Ce problème (encore ouvert) est appelé _problème du logarithme discret_. 

En 1976, Diffie et Hellman partent de ce constat pour proposer un protocole d'échange de clés encore utilisé de nos jours. Le principe est qu'Alice et Bob souhaitent chiffrer leurs échanges, mais qu'ils ne peuvent pas se retrouver pour échanger de clé de chiffrement. Ils vont donc échanger des informations sur un canal non sécurisé pour créer une clé de chiffrement commune, mais de sorte qu'un attaquant passif (quelqu'un qui écoute leurs communications, mais sans rien envoyer) ne sera pas capable de retrouver la clé de chiffrement. 

## 1. Exponentiation rapide

On s'intéresse tout d'abord à l'exponentiation. 

Ecrire un algorithme `exp_naif(a,n)` qui prend en argument $a$ un élément d'un groupe $G$ et $n$ un entier positif, puis qui calcule $a^n$. 

_Remarque : dans Sage, les groupes sont en général écrits en notation multiplicative, donc le produit se fait grâce à l'opérateur `*`._

_Remarque 2 : pour cet exercice, n'utilisez pas les fonctions Sage `pow`, `^` ou `**`, le but est de les recoder pour étudier leur efficacité !_

Quelle est la complexité de cet algorithme ? Plus exactement, combien de multiplication ont été effectuées ? 

En cryptographie, on travaille souvent avec des exposants monstrueux (c'est à dire que $n$ est plus grand que le nombre d'atomes dans l'univers !), aussi on ne peut pas se permettre d'avoir une complexité linéaire pour ce genre d'algorithmes. On va donc devoir utiliser une autre méthode. 

L'algorithme présenté ici est appelé "square and multiply". Il en existe plusieurs variantes, mais on n'en donne ici qu'une première implémentation peu optimisée. 

On souhaite calculer $a^n$, où $a$ est un élément d'un groupe $G$ fini et $n$ est un exposant très grand. L'idée de l'algorithme square and multiply est d'écrire la décomposition en binaire de $n$ :

$$n = \sum\limits_{i=0}^k 2^i n_i,\quad n_i\in\{0,1\}$$

et de remarquer que :

$$a^n = \prod\limits_{i=0}^k a^{2^i \times n_i}$$

Ensuite, on peut calculer $a^{2^i}$ avec seulement $i$ multiplications : en effet $a^{2^i} = (\dots((a^2)^2)^2\dots)^2$.

Avec cela, on n'a plus qu'à calculer les $a^{2^i}$, et à les multiplier entre eux. 

L'algorithme se présente alors comme suit : 



**INPUT** : `a` un élément d'un groupe fini, `n` un entier naturel.

**OUTPUT** : `a^n`

Décomposer $n$ en $n = \sum\limits_{i=0}^k 2^i n_i$

`liste_puissances` $\gets$ `[a]`    # on calcule la liste des puissances telle que `liste_puissances[i]`$ = a^{2^i}$

`a_2i` $\gets$ `a`

Pour $i$ allant de $1$ à $k$:

- `a_2i` $\gets$ `a_2i` * `a_2i`

- ajouter `a_2i` à `liste_puissances`

`res` $\gets 1$

Pour $i$ allant de $0$ à $k$:

- si $n_i = 1$ : `res` $\gets$ `res` * `liste_puissances[i]`

Retourner `res`


Combien de multiplications (dans $G$) sont nécessaires pour cet algorithme ? 

## 2. Echange de clés de Diffie-Hellman

On présente maintenant le protocole proposé par Diffie-Hellman.

Alice et Bob commencent par se mettre d'accord sur un groupe $G$ d'ordre $n$ et sur $g$ un générateur de $G$, qu'ils peuvent envoyer publiquement.

Alice choisit un entier $m_A <n$ et calcule $A = g ^{m_A}$, puis envoie $A$ à Bob.

De son côté Bob choisit aussi un nombre $m_B < n$ aléatoire et calcule $B = g^{m_B}$, puis l'envoie à Alice. 

Alice et Bob vont alors respectivement calculer $B^{m_A}$ et $A^{m_B}$. 

Montrez dans la cellule suivante qu'Alice et Bob ont bien calculé la même clé :

Comme on l'a dit plus haut, si Eve écoute les conversations entre Alice et Bob, elle connaîtra les éléments $A,B$, mais elle ne sera (a priori) pas capable de calculer les exposants $m_A$ et $m_B$, et donc de retrouver la clé partagée entre Alice et Bob. 

Pour ce protocole, vous aurez besoin de trois fonctions : 

- `trouve_gen` qui génère un groupe $G$ et un générateur $\alpha$ selon les besoins
    
- `partial_key_gen` qui permet à Alice (resp. Bob) de générer le secret $m_A$ (resp. $m_B$) et la clé partielle A (resp. B) 
    
- `full_key_gen` qui permet à Alice (resp. Bob) de calculer la clé publique avec les paramètres $m_A$ et $B$ (resp. $m_B$ et $A$)

### Première implémentation

Essayer d'implémenter ce protocole avec $G = (\mathbb Z/n \mathbb Z, +)$. 

Cette manière de faire n'offre toutefois aucune sécurité. Pouvez-vous trouver une manière de retrouver la clé à partir des informations publiques (c'est à dire $A,B$ et $g$) ?

### Deuxième tentative

Lorsque $p$ est un nombre premier, on sait que $(\mathbb Z/p\mathbb Z)^\times$ est cyclique. 

Essayez à nouveau d'implémenter le protocole avec ce groupe. Pensez-vous que cette implémentation soit sûre ? 

_Note : si vous le souhaitez, vous pouvez implémenter l'algorithme dans le cas général où $(G,*)$ est un groupe cyclique._

## 3. Une attaque sur le logarithme discret

### Une attaque dans un cas particulier

Supposons dans un premier temps que $G$ est un groupe cyclique de cardinal $2^{r+1}$, et $g$ est un générateur de $G$. On va montrer que dans ce cas particulier le problème du logarithme discret se résout avec $O(r^2)$ opérations sur $G$.

Soit un élément $\alpha \in G$. On souhaite trouver $n \in \mathbb N$ tel que $\alpha = g^n$. 

L'idée de l'attaque présentée ici est de déterminer tour à tour chacun des bits de $n$.

On commence par écrire la décomposition en binaire $n = \sum\limits_{i=0}^r 2^i n_i,\, n_i\in\{0,1\}$. 

Si $n_0 = 0$, alors on sait que $n = 2 \times \sum\limits_{i=1}^r 2^{i-1} n_i = 2n'$. En particulier, on obtiens :

$$\alpha^{2^{r}} = (g^n)^{2^{r}} = g^{2n' 2^{r}} = g^{n' 2^{r+1}} = (g^{2^{r+1}})^{n'} = e_G^{n'} = e_G$$

car $g^{\#G} = e_G$. 

À l'inverse, si $n_0 \neq 0$, on a $\alpha^{2^{r}} = g^{2^{r}} \neq e_G$ comme $g$ est un générateur de $G$. 

On en déduit que $n_0=0\Leftrightarrow \alpha^{2^{r}}= e_G$. Donc, en calculant $\alpha^{2^{r}}$, on apprend donc le dernier bit de $n$. 

Comment obtenir les autres bits de $n$ ? 

On considère alors $\alpha_1 = \alpha * g^{-n_0}$. C'est un élément de $G$ et on a $\alpha_1 = g^{\sum_{i=1}^r 2^i n_i}$ (on a enlevé $n_0$), donc en particulier d'après la discussion ci-dessus $\alpha_1^{2^{r}} = e_G$. 

Comme au dessus, on peut montrer que $\alpha_1 ^{2^{r-1}} = e_G \Longleftrightarrow n_1 = 0$. 

On pose alors $\alpha_2 = \alpha_1* g^{- 2n_1} = \alpha * g^{-2n_1 - n_0}$, et on a $\alpha_2 ^{2^{r-2}} = e_G \Longleftrightarrow n_2 = 0$.

On réitère cette opération jusqu'à obtenir $n_r$, et on a alors la décomposition binaire de $n$.

Compléter l'algorithme suivant pour calculer efficacement le logarithme discret d'un élément $g$ :

In [ ]:
# INPUT : - g un élément générateur d'un groupe G (dont le cardinal est 2^(r+1))
#         - alpha un élément de G
#         - r un entier tel que #G = 2^(r+1)
# OUTPUT : n un entier tel que g = alpha^n

def log_discret_pow_2(g, alpha, r):
    ni = [0 for i in range(r)] # pour un entier 0 <= i< r, on veut ni[i] = n_i comme défini plus haut (le i-ième bit de n)
    alpha_i =  ????????? # comment initialiser alpha_i ?
    for i in range(r+1):
        # remplissez ici les calculs nécessaires 
        
        
        if ???????? :
            ni[i] = 1
        else :
            ni[i] = 0
    
    # une fois que tous les bits de n ont été récupérés, on retrouve n :
    n = 0
    for i in range(r):
        n += ni[i] * 2^i
    
    return n

Vous pouvez tester votre programme sur  $(\mathbb Z/p\mathbb Z)^\times$, avec $p=17,\, 257,\, 65\, 537$ pour vérifier si il fonctionne bien. 

### Un cas un peu plus général

Si on a $\#G = 3^r$, les choses se compliquent un peu. 

En notant $\alpha = g^n$ avec $n = \sum\limits_{i=0}^r 3^i n_i,\, n_i\in\{0,1,2\}$, la décomposition en base $3$ de $n$, on a :

$$\alpha^{3^{r-1}} = (g^{3n' + n_0})^{3^{r-1}} = g^{3^{r-1}n_0}$$

Or ici, $n_0$ peut prendre trois valeurs différentes : $0,1,2$. Ainsi on n'a pas directement la valeur de $n_0$ comme dans le cas précédent. On doit alors comparer $\alpha^{3^{r-1}}$ avec $e_G$, puis avec $g^{3^{r-1}}$ et avec $g^{3^{r-1}\times 2}$ pour obtenir $n_0$. 

On pose ensuite de la même manière $\alpha_1 = \alpha * g^{-n_0}$, et on réitère le raisonnement. 

On va maintenant essayer de généraliser cette approche:

Supposons maintenant que $\#G = \ell^{r+1}$, où $\ell$ est un nombre premier. 

Adapter le programme précédent dans ce cas. 

_Indice : à chaque étape vous aurez besoin de comparer $\alpha_i ^{\ell^{r-i}}$ avec $g^{\ell^{r}},g^{\ell^{r}\times 2},\dots, g^{\ell^{r}\times (\ell-1)}$, donc autant les calculer une fois pour toutes et les stocker dans une liste `g_pow`_

In [ ]:
# INPUT : - g un élément générateur d'un groupe G (dont le cardinal est l^(r+1))
#         - alpha un élément de G
#         - l un nombre premier et r un entier tel que #G = l^(r+1)
# OUTPUT : n un entier tel que g = alpha^n

def log_discret_pow_l(g, alpha, l, r):
    ni = [0 for i in range(r)] # pour un entier 0 <= i< r, on veut ni[i] = n_i le i-ième bit de n en base l
    alpha_i =  ????????? # comment initialiser alpha_i ?
    
    gr= g^(l^r)
    g_pow = [gr^i for i in range(l)] # comme dans l'indication
    
    for i in range(r+1):
        # remplissez ici les calculs nécessaires 
        
        
        
        ni[i] = ????? # déterminer n_i avec 0<= n_i < l
        
    
    # une fois que tous les bits de n ont été récupérés, on retrouve n :
    n = 0
    for i in range(r):
        n += ni[i] * l^i
    
    return n

### Le cas général

Supposons qu'on a maintenant que $G$ est un groupe cyclique avec $\#G = N = \prod\limits_{j = 1}^s \ell_j^{r_j}$ et $g$ un générateur de $G$. 

Soit $\alpha = g^n \in G$, et cherchons comment retrouver $n$.

On peut montrer que $G \cong \mathbb Z/N \mathbb Z \cong \prod\limits_{j = 1}^s \mathbb Z/\ell_j^{r_j}\mathbb Z$ d'après le théorème des restes chinois. Ainsi, toujours d'après le théorème des restes chinois, si on arrive à calculer $(n \mod \ell_j^{r_j})$ pour tout $j$, on peut retrouver $n$. 

Pour un entier $ 1 \leqslant k \leqslant s$, on commence par noter $N_k := \prod\limits_{j = 1, j \neq k}^s \ell_j^{r_j}$. On sait alors que $g_k := g^{N_k}$ est un élément d'ordre $\ell_k^{r_k}$, et que $\alpha_k := \alpha^{N_k}$ est dans le groupe engendré par $g_k$. On peut donc appliquer l'algorithme de la partie précédente pour déterminer l'entier $n_k$ tel que $\alpha_k = g_k^{n_k}$. 

On en déduit alors que $N_k \times n \equiv n_k \mod \ell_k^{r_k}$. Il suffit donc d'inverser $N_k$ modulo $\ell_k^{r_k}$ pour trouver $(n \mod \ell_k^{r_k})$. 

On effectue cela pour tous les $\ell_k$, puis on utilise le théorème des restes chinois pour déterminer $n$. 

Ecrivez un programme pour calculer le logarithme discret dans le cas général. 

_Remarque : pour cela vous devrez appeler la fonction `log_discret_pow_l` de l'exercice précédent. Pour retrouver la valeur de n il faut faire appel au théorème des restes chinois codé dans le TP précédent, donc on ne demande ici que de calculer la valeur de $n \mod \ell_k^{r^k}$._

In [ ]:
# INPUT : - g un générateur de G
#         - alpha un élément de G
#         - li et ri deux listes d'entiers telles que #G = prod(li[j]^ri[j] for j in range(s))
# OUTPUT : une liste nk = [n_1, ... , n_s] d'entiers tels que g^n = alpha et pour tout k : n = n_k mod (l_k)^(r_k)

def log_dicret_gen(g, alpha, li , ri, s):
    N = prod([li[j]^ri[j] for j in range(s)])
    Nk = [N//(li[j]^ri[j]) for j in range(s)]
    
    # Compléter le programme 
    
    

_Remarque : cette manière de faire est effectivement la plus efficace dans le cas général (modulo quelques optimisations) mais reste extrêmement lente si un des $\ell_j$ est un nombre premier "grand"._

_Plus exactement, remarquez que le programme `log_discret_pow_l` nécessite de calculer les $(g^{\ell^r})^i$ pour $0<i<\ell$, ce qui implique une complexité en $O(r\ell)$. En notant $L = \max_i(\ell_i)$, la complexité de l'algorithme général est donc en $O(\log(\#G) \cdot L)$. Si on s'arrange pour que $L$ soit très grand, cette attaque devient donc inefficace._

_Pour cette raison, les protocoles à la Diffie-Hellman sont toujours considérés comme sûrs, mais seulement pour des groupes de cardinal bien choisi._ 